<a href="https://colab.research.google.com/github/NarmadhaJayasankar/Covidapp/blob/feature/EW_GA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install requests pandas sqlalchemy


In [4]:
from sqlalchemy import create_engine
import requests
import pandas as pd
from sqlalchemy import create_engine

# Rest of your code remains the same


In [7]:
import requests
import pandas as pd
from sqlalchemy import create_engine
json_api_url = "https://connectors.windsor.ai/all?api_key=a08773c49de68206ec3bcbff003f45f1e4d3&date_preset=last_7d&fields=account_name,campaign,date,sessions,source"
response = requests.get(json_api_url)
data_json = response.json()
df = pd.DataFrame(data_json)
#converting json data into dataframe
# SQL Server connection parameters
db_username = 'sa'
db_password = 'Welcome@#123'
db_server = 'CESIT-LAP-0339'
db_name = 'your_database_name'
engine = create_engine(f"mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server")
table_name = 'your_table_name'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)
print(f"Data successfully transferred to SQL Server table '{table_name}'")


ModuleNotFoundError: ignored

In [9]:
pip install pyodbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.8/334.8 kB 5.0 MB/s eta 0:00:00


In [14]:
odbcinst -q -d -n "ODBC Driver 17 for SQL Server"


SyntaxError: ignored

In [13]:
import requests
import pyodbc
import pandas as pd
import json
from datetime import datetime
start_time = datetime.now()
api_url = "https://connectors.windsor.ai/all?api_key=a08773c49de68206ec3bcbff003f45f1e4d3&date_preset=last_7d&fields=account_name,campaign,date,sessions,source"
database = 'Narma'
username = 'sa'
password = 'Welcome@#123'
dsn = '15.0.2000'
db_connection_string = f'DSN={dsn};DATABASE={database};UID={username};PWD={password}'
connection = pyodbc.connect(db_connection_string)
cursor = connection.cursor()
response = requests.get(api_url)
results = response.json()
json_objects=[]
if response.status_code == 200:
    for row in results["data"]:
        json_objects.append(row)
    json_array = json.dumps(json_objects)
    data_list = json.loads(json_array)
    column_data_types = {}
    for record in data_list:
        for key, value in record.items():
            if key not in column_data_types:
                column_data_types[key] = type(value).__name__
    df = pd.DataFrame(data_list)
    df.isna()
    df.fillna(0, inplace=True)
    table_name = "googleanalytics_sessions"
    cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'")
    column_data_types = {row[0]: row[1] for row in cursor.fetchall()}
    df_copy=df.copy()
    for column_name, data_type in column_data_types.items():
        if data_type == 'integer':
            df_copy[column_name] = df_copy[column_name].astype(int)
        elif data_type == 'text':
            df_copy[column_name] = df_copy[column_name].astype(str)
    print(df_copy.columns)
    for index, row in df_copy.iterrows():
        columns = ', '.join(df_copy.columns)
        placeholders = ', '.join(['?' for _ in df_copy.columns])
        values = [row[column_name] for column_name in df_copy.columns]
        insert_statement = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(insert_statement, values)
    connection.commit()
    pass
else:
    print(f"Error: {response.status_code} - {response.text}")
cursor.close()
connection.close()
end_time = datetime.now()
elapsed_time = end_time-start_time
print(f"Data got inserted to the table: {table_name} with elapsed time: {elapsed_time}")

InterfaceError: ignored

In [15]:
import pyodbc
import requests
import pandas as pd
import json
from datetime import datetime

start_time = datetime.now()
api_url = "https://connectors.windsor.ai/all?api_key=a08773c49de68206ec3bcbff003f45f1e4d3&date_preset=last_7d&fields=account_name,campaign,date,sessions,source"
database = 'Narma'
username = 'sa'
password = 'Welcome@#123'
dsn = 'SQLServer_Local'  # Update with your DSN name

# Create the ODBC connection string
db_connection_string = f'DSN={dsn};DATABASE={database};UID={username};PWD={password}'

# Connect to the SQL Server
try:
    connection = pyodbc.connect(db_connection_string)
    cursor = connection.cursor()
except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")
    exit()

# Fetch data from the JSON API
response = requests.get(api_url)
results = response.json()

# Check if the response is successful
if response.status_code == 200:
    # Convert JSON data to DataFrame
    df = pd.DataFrame(results.get("data", []))

    # Handle NaN values
    df.fillna(0, inplace=True)

    # Fetch column data types from the SQL Server table
    table_name = "googleanalytics_sessions"  # Update with your table name
    cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'")
    column_data_types = {row[0]: row[1] for row in cursor.fetchall()}

    # Adjust DataFrame column types based on SQL Server table
    for column_name, data_type in column_data_types.items():
        if data_type == 'int':
            df[column_name] = df[column_name].astype(int)
        elif data_type == 'nvarchar':
            df[column_name] = df[column_name].astype(str)

    # Insert data into the SQL Server table
    for index, row in df.iterrows():
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for _ in df.columns])
        values = [row[column_name] for column_name in df.columns]
        insert_statement = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(insert_statement, values)

    # Commit the changes
    connection.commit()

    print(f"Data successfully inserted into the table: {table_name}")
else:
    print(f"Error: {response.status_code} - {response.text}")

# Close the cursor and connection
cursor.close()
connection.close()

end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")


Error connecting to the database: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found and no default driver specified (0) (SQLDriverConnect)')


NameError: ignored

In [1]:
import pyodbc
import requests
import pandas as pd
import json
from datetime import datetime

start_time = datetime.now()
api_url = "https://connectors.windsor.ai/all?api_key=a08773c49de68206ec3bcbff003f45f1e4d3&date_preset=last_7d&fields=account_name,campaign,date,sessions,source"
database = 'Narma'
username = 'sa'
password = 'Welcome@#123'
dsn = 'SQLServer_Local'  # Update with your DSN name

# Create the ODBC connection string
db_connection_string = f'DSN={dsn};DATABASE={database};UID={username};PWD={password}'

# Initialize cursor variable outside the try block
cursor = None

# Connect to the SQL Server
try:
    connection = pyodbc.connect(db_connection_string)
    cursor = connection.cursor()
except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")
    exit()

# Fetch data from the JSON API
response = requests.get(api_url)
results = response.json()

# Check if the response is successful
if response.status_code == 200:
    # Convert JSON data to DataFrame
    df = pd.DataFrame(results.get("data", []))

    # Handle NaN values
    df.fillna(0, inplace=True)

    # Fetch column data types from the SQL Server table
    table_name = "googleanalytics_sessions"  # Update with your table name
    cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'")
    column_data_types = {row[0]: row[1] for row in cursor.fetchall()}

    # Adjust DataFrame column types based on SQL Server table
    for column_name, data_type in column_data_types.items():
        if data_type == 'int':
            df[column_name] = df[column_name].astype(int)
        elif data_type == 'nvarchar':
            df[column_name] = df[column_name].astype(str)

    # Insert data into the SQL Server table
    for index, row in df.iterrows():
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for _ in df.columns])
        values = [row[column_name] for column_name in df.columns]
        insert_statement = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(insert_statement, values)

    # Commit the changes
    connection.commit()

    print(f"Data successfully inserted into the table: {table_name}")
else:
    print(f"Error: {response.status_code} - {response.text}")

# Close the cursor and connection
if cursor:
    cursor.close()
connection.close()

end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")


Error connecting to the database: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found and no default driver specified (0) (SQLDriverConnect)')


AttributeError: ignored

In [11]:
import requests
import pyodbc
import pandas as pd
import json
from datetime import datetime

start_time = datetime.now()
api_url = "https://connectors.windsor.ai/all?api_key=a08773c49de68206ec3bcbff003f45f1e4d3&date_preset=last_7d&fields=account_name,campaign,date,sessions,source"
database = 'Narma'
username = 'sa'
password = 'Welcome@#123'
server = ''
table_name = "googleanalytics_sessions"

# Specify the ODBC driver and create the connection string
db_connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Connect to the SQL Server
connection = pyodbc.connect(db_connection_string)
cursor = connection.cursor()

# Fetch data from the JSON API
response = requests.get(api_url)
results = response.json()

# Check if the response is successful
if response.status_code == 200:
    # Convert JSON data to DataFrame
    df = pd.DataFrame(results.get("data", []))

    # Handle NaN values
    df.fillna(0, inplace=True)

    # Fetch column data types from the SQL Server table
    cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'")
    column_data_types = {row[0]: row[1] for row in cursor.fetchall()}

    # Adjust DataFrame column types based on SQL Server table
    for column_name, data_type in column_data_types.items():
        if data_type == 'int':
            df[column_name] = df[column_name].astype(int)
        elif data_type == 'nvarchar':
            df[column_name] = df[column_name].astype(str)

    # Insert data into the SQL Server table
    for index, row in df.iterrows():
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for _ in df.columns])
        values = [row[column_name] for column_name in df.columns]
        insert_statement = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(insert_statement, values)

    # Commit the changes
    connection.commit()

    print(f"Data successfully inserted into the table: {table_name}")
else:
    print(f"Error: {response.status_code} - {response.text}")

# Close the cursor and connection
cursor.close()
connection.close()

end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")


Error: ignored